# Scheduler Local Test


## Setup


* `make start-scheduler` 
* `make start-envoy`
* `make start-agent`
* `make start-mlserver`
* `make start-rclone`

## No Auth Example

In [172]:
!grpcurl -d '{"model":{"name":"iris","version":"2","uri":"gs://seldon-models/mlserver/iris",\
         "requirements":["sklearn"],\
         "memoryBytes":500,\
         "replicas":1}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [173]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [174]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x5583b09e74f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x5583b09e74f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Sat, 27 Nov 2021 10:35:38 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1425
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"8536bc4d-8e25-484b-8a2d-a5abb944bbb7","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [175]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [176]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "version": "2",
  "serverName": "mlserver",
  "modelReplicaState": {
    "0": "Loaded"
  }
}


In [177]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [178]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [179]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

ERROR:
  Code: FailedPrecondition
  Message: Failed to find model iris


In [180]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "1000"
    }
  ]
}


In [181]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x558ffc62b4f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x558ffc62b4f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)







* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


## Inline RClone Config Example

Before running this 
  * install minio in a kind cluster with Ansible.
  * Update ip address for exposed minio ip address in example below

In [195]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
           storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://172.18.255.1:9000\"}}" \
     } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [183]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [184]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x56019c96e4f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x56019c96e4f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Sat, 27 Nov 2021 10:36:08 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1443
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"b2837aac-90f4-40d0-9323-f4ad1450044f","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [185]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [186]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


## K8S Secret RClone Config Example

 * Update endpoint for minio below

In [187]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://172.18.255.1:9000

Overwriting minio-secret.yaml


In [188]:
!kubectl apply -f minio-secret.yaml

secret/minio-secret configured


In [189]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
      storageSecretName: "minio-secret" \
      } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [190]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [191]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x559a5e7b34f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x559a5e7b34f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Sat, 27 Nov 2021 10:36:22 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1410
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"e017998f-2200-4d32-b6c2-1ea6323a8e4e","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [192]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [193]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}
